### Tersor....Board....ing 

#### Watching the [introduction video](https://www.youtube.com/watch?v=eBbEDRsCmv4&t=1105s) [Tutorial](https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial)

If (Tensorboard == useful): 
    Install tensorflow https://www.tensorflow.org/install   
else 
    Close this notebook

#### start the tensorboard : 

> /Prometheus/src$ tensorboard --logdir=./logs_ann/
> 
> TensorBoard 1.8.0 at __http://*****:6006__ (Press CTRL+C to quit)
> 

Tips: if you open the 6006 url link but find nothing, calm down, double check the path. Wrong folder will give you empty board 


####  Wrong code could too, so the code begins here

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from os import listdir
from tensorflow.python.client import timeline

import calendar
import time
import arrow
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU,Input
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.losses import mean_squared_error

## import data and tailor it for use

In [ ]:
tf.reset_default_graph()

import pickle
with open("data", "rb") as f:
    df_X = pickle.load(f)
    df_Y = pickle.load(f)

shift_iter = 24
X = df_X.values[shift_iter + 1:-shift_iter - 1]
Y = df_Y.values[shift_iter + 1:-shift_iter - 1]
# print(X.shape)
# print(Y.shape)
# print(list(df_X),list(df_Y))

ratio = 0.75  # for spliting train - test set
num_train = round(X.shape[0] * ratio)
x_train = X[:num_train]
y_train = Y[:num_train]
x_test = X[num_train:]
y_test = Y[num_train:]

x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

num_x_signals = X.shape[1]
num_y_signals = Y.shape[1]


In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    You need to prepare the data before running this x_train_scaled , y_train_scaled
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)

            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx + sequence_length]
            y_batch[i] = y_train_scaled[idx:idx + sequence_length]

        yield (x_batch, y_batch)

        
class TensorBoard_ValidateScalar(TensorBoard):
    def __init__(self, log_dir='./logs_for_tensorboard', **kwargs):
        
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super().__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super().set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary(value=[tf.Summary.Value(tag=name, simple_value=value)])
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        train_logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super().on_epoch_end(epoch, train_logs) # default send into scalars 
        
    def on_train_end(self, logs=None):
        super().on_train_end(logs)
        self.val_writer.close()

## Neural network on Keras (Tensorflow)

In [ ]:
model = Sequential()

model.add(Dense(num_x_signals ,
              activation='hard_sigmoid',
              name = "hidden",
              input_shape=(None, num_x_signals,)))

model.add(Dense(num_y_signals, 
                activation='linear',
                name = "output"))
run_metadata= tf.RunMetadata()
model.compile(
          loss = mean_squared_error,
          optimizer = Adam(lr=1e-2),
          metrics = ['accuracy','mae','mean_squared_error','mape'], # this part will show up in tensorboard
          options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE), 
          run_metadata= tf.RunMetadata(), # these two show up in Chrome Trace
             )

model.summary()


In [ ]:
log_dir="./logs_tensorboard_demo/{}".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))
print(log_dir)


model.fit_generator(generator = batch_generator(batch_size=200, sequence_length=168) ,
                    epochs=10,
                    verbose = 1,
                    steps_per_epoch=80,
                    validation_data=validation_data,
                    callbacks=[
                           TensorBoard_ValidateScalar(
                           log_dir,
                           histogram_freq = 1,
                           write_graph = True, # visualize the graph in TensorBoard
                           #write_images = True, #  write model weight&bias to visualize as image in TensorBoard
                           write_grads = True, # visualize training gradient histograms in TensorBoard
                            )])
print('Run `tensorboard --logdir=%s` to see the results.' % log_dir)

### Tracing
####  Use chrome and open link chrome://tracing , load trace file named as timeline1.ctf.json to check the data running time and flow direction.

In [ ]:
trace = timeline.Timeline(step_stats = run_metadata.step_stats)
with open(log_dir+'/timeline1.ctf.json','w') as f:
    f.write(trace.generate_chrome_trace_format())

## Neural network on Tensorflow
tf.summary.

* scalar(...): Outputs a Summary protocol buffer containing a single scalar value.
        
* histogram(...): Outputs a Summary protocol buffer with a histogram.
        
* audio(...): Outputs a Summary protocol buffer with audio.
* image(...): Outputs a Summary protocol buffer with images.
* text(...): Summarizes textual data.
        
* merge(...): Merges summaries
* merge_all(...): Merges all summaries collected in the default graph.
        
* tensor_summary(...): Outputs a Summary protocol buffer with a serialized tensor.proto.
* get_summary_description(...): Given a TensorSummary node_def, retrieve its SummaryDescription.

``` 
    -------- You can start code like -------- 
    with tf.summary.FileWriter(log_dir) as writer: 
        do whatever
       
   --------OR --------
   writer = tf.summary.FileWriter(log_dir)
   do whatever 
   writer.close()
      
```

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import arrow

log_dir="./logs_tensorboard_demo/{}".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))

# making up some random dataset
x_data = np.linspace(-1,1,300)[:,np.newaxis] # flatten your arrary into list
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data)-0.5+noise

# Clears the default graph stack and resets the global default graph 
tf.reset_default_graph()

# tf.summary.image('input',tf.reshape(x_data,[1,300,1,1]))
with tf.name_scope('input_layer'):
    x = tf.placeholder(tf.float32,[None,1],name='x_input')
    y = tf.placeholder(tf.float32,[None,1],name='y_input')

    
with tf.name_scope('hidden_layer'):
    with tf.name_scope('W'):
        W1 = tf.Variable(tf.random_normal([1,10]))
        tf.summary.histogram('hidden-weight',W1)

    with tf.name_scope('B'):
        b1 = tf.constant(0.5,shape=[1,10])
        tf.summary.histogram('hidden-bias',b1)

    with tf.name_scope('Activation'):
        a = tf.matmul(x,W1)+b1
        tf.summary.histogram('hidden-pre-active',a)

    hidden = tf.nn.softmax(a)
    tf.summary.histogram('hidden-activation',hidden)

with tf.name_scope('output_layer'):
    with tf.name_scope('W'):
        W2 = tf.Variable(tf.random_normal([10,1]))
        tf.summary.histogram('output-weight', W2)

    with tf.name_scope('B'):
        b2 = tf.constant(0.1, shape=[1, 1])
        tf.summary.histogram('output-bias', b2)

    with tf.name_scope('Activation'):
        a = tf.matmul(hidden, W2) + b2
        tf.summary.histogram('output-pre-active', a)

        output = tf.nn.relu(a)
        tf.summary.histogram('output-activation', output)


with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(y-output),reduction_indices=[1]))
    tf.summary.scalar('loss',loss)


with tf.name_scope('train'):
    train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)


init = tf.global_variables_initializer()
merge = tf.summary.merge_all()
with tf.Session() as sess:
    
    sess.run(init)
    writer = tf.summary.FileWriter(log_dir,sess.graph)

    for i in range(500) :
        sess.run(train,feed_dict={x:x_data,y:y_data})
        result = sess.run(merge,feed_dict={x:x_data,y:y_data})
        writer.add_summary(result,i)
        
    writer.close()
print(log_dir)    

### Embedding projector in Tensorboard [Demo](https://projector.tensorflow.org/)
Demo from official tensorboard 
https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial/blob/master/mnist.py 


In [ ]:
import os
import os.path
import tensorflow as tf
import arrow
LOGDIR="./tmp/mnist_tutorial/{}/".format(arrow.now().format('YYYY-MM-DD-HH-MM-SS'))

# You need to download these two files from the github link and put in the current folder same as notebook 
LABELS = os.path.join(os.getcwd(), "labels_1024.tsv")
SPRITES = os.path.join(os.getcwd(), "sprite_1024.png")

### MNIST EMBEDDINGS ###
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR + "data", one_hot=True)

### Get a sprite and labels file for the embedding projector ###

if not (os.path.isfile(LABELS) and os.path.isfile(SPRITES)):
    print("Necessary data files were not found. Run this command from inside the "
          "repo provided at "
          "https://github.com/dandelionmane/tf-dev-summit-tensorboard-tutorial.")
    os.exit(1)


# shutil.copyfile(LABELS, os.path.join(LOGDIR, LABELS))
# shutil.copyfile(SPRITES, os.path.join(LOGDIR, SPRITES))


def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act


def mnist_model(learning_rate, use_two_fc, use_two_conv, hparam):
    # reset and clean the default graph in stack
    tf.reset_default_graph()
    with tf.Session() as sess:
        
        # Setup placeholders, and reshape the data
        x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
        x_image = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', x_image,max_outputs=10)
        y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

        if use_two_conv:
            conv1 = conv_layer(x_image, 1, 32, "conv1")
            conv_out = conv_layer(conv1, 32, 64, "conv2")
        else:
            conv_out = conv_layer(x_image, 1, 16, "conv")

        flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

        if use_two_fc:
            fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
            relu = tf.nn.relu(fc1)
            embedding_input = relu
            tf.summary.histogram("fc1/relu", relu)
            embedding_size = 1024
            logits = fc_layer(relu, 1024, 10, "fc2")
        else:
            embedding_input = flattened
            embedding_size = 7 * 7 * 64
            logits = fc_layer(flattened, 7 * 7 * 64, 10, "fc")

        with tf.name_scope("loss"):
            xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                    logits=logits, labels=y), name="loss")
            tf.summary.scalar("loss", xent)

        with tf.name_scope("train"):
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

        with tf.name_scope("accuracy"):
            correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar("accuracy", accuracy)

        summ = tf.summary.merge_all()

        embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
        assignment = embedding.assign(embedding_input)
        
        # The embedding projector will read the embeddings from your model checkpoint file
        saver = tf.train.Saver()

        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter(LOGDIR + hparam)
        writer.add_graph(sess.graph)

        config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
        embedding_config = config.embeddings.add()
        embedding_config.tensor_name = embedding.name
        embedding_config.sprite.image_path = SPRITES
        embedding_config.metadata_path = LABELS
        # Specify the width and height of a single thumbnail.
        embedding_config.sprite.single_image_dim.extend([28, 28])
        tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

        for i in range(2001):
            batch = mnist.train.next_batch(100)
            if i % 5 == 0:
                [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
                writer.add_summary(s, i)
            if i % 500 == 0:
                sess.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
                saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
            sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})
    

def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)


def main():
    # You can try adding some more learning rates
    for learning_rate in [1E-3]: #, 1E-4
        # Include "False" as a value to try different model architectures
        for use_two_fc in [False]:
            for use_two_conv in [False]: #, True
                # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2")
                hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
                print('Starting run for %s' % hparam)

                # Actually run with the new settings
                mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)
    print('Done training!')
    print('Run `tensorboard --logdir=%s` to see the results.' % LOGDIR)



if __name__ == '__main__':
    main()


# Ref 
- [t-SNE](https://distill.pub/2016/misread-tsne/)

    A popular method for exploring high-dimensional data is something called t-SNE, introduced by van der Maaten and Hinton in 2008. 
    
    The technique has become widespread in the field of machine learning, since it has an almost magical ability to create compelling two-dimensonal “maps” from data with hundreds or even thousands of dimensions. 